Кобзарь О.С. Горидько К.А. Хабибуллин Р.А.

последнее изменение 03.09.2019

## TODO сделать крутой аналитичейски dashboard с возможность настройки под различные исследования

# Работа с исходными данными исследований


Сразу дадим названия файликов с данными калибровочными и исследуемыми и пометку в названии сгенерируемого файлика excel

In [ ]:
file_name_with_regime_data = "T72.trd"
file_name_with_corr_data = "T66.trd"
changed_mark = "31_08_2019" 

Импорт необходимых модулей

In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from _plotly_future_ import v4_subplots
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.graph_objs as go

from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
init_notebook_mode(connected=True)
import os
from datetime import date

Проинициализируем все функции для обработки данных и построения графиков

In [ ]:
def edit_new_trd_file(file):
    del file[72]
    del file[0]
    del file[1]
    nedeed_columns = file.columns[2:]
    for i in nedeed_columns:
        ost = i % 2
        if ost == 0 and i != 72:
            sensor_name = file[i][0]
        else:
            file = file.rename(columns={i:sensor_name})
            if i != 72:
                del file[i-1]

    #del file['PLC_PRG.Diaf_Number']
    file["Время"] = file[2] + " " + file[3]
    file["Время"] = pd.to_datetime(file["Время"], dayfirst = True)
    file.index = file["Время"]
    del file[2]
    del file[3]
    del file["Время"]

    file = file.drop(file.index[0:4])  # удаляем первую строчку со значениями
    return file

def func_to_row_data(min_val, max_val, row_val, multiplier):
    result = (min_val + (max_val - min_val) * row_val / 20000) * multiplier
    result.iloc[-2] = row_val.iloc[-2]
    result.iloc[-1] = row_val.iloc[-1]
    result.iloc[5] = ((max_val[5] - min_val[5]) * row_val[5] / 20000) * multiplier[5]
    return result 

def trace(data_x, data_y, namexy, chosen_mode = 'lines+markers'):
    tracep = go.Scattergl(
        x=data_x,
        y=data_y,
        name=namexy,
        mode=chosen_mode
    )
    return tracep


def plot_func(plot_title_str, filename_str):
    layout = dict(title=plot_title_str)

    fig = dict(data=data, layout=layout)
    
    #fig = make_subplots(rows=8, cols=1)


    
    plot(fig, filename=filename_str)
    #iplot(fig)

def plot_subplots(data_traces, plot_title_str, filename_str, two_equal_subplots = False):
    if two_equal_subplots:
        items_in_one_subplot = int(len(data_traces) )
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
        for i in range(items_in_one_subplot):
            fig.append_trace(data_traces[i], row = 1, col = 1)
            fig.append_trace(data_traces[i], row = 2, col = 1)
    else:
        fig = make_subplots(rows=len(data_traces), cols=1, shared_xaxes=True, vertical_spacing=0.02)
        for i in range(len(data_traces)):
            fig.append_trace(data_traces[i], row = i + 1, col = 1)
    
    
    
    plot(fig, filename=filename_str)
    

def create_traces_list_by_num(data_x_values, data_y, num_y_list):
    trace_list = []
    for i in num_y_list:
        namexy = data_y.get_saved_parameter_name_by_number(i)
        this_trace = trace(data_x_values, data_y.get_saved_values_by_number(i), namexy)
        trace_list.append(this_trace)
    return trace_list

def create_traces_list_for_all_columms(data_frame, chosen_mode = 'lines+markers'):
    trace_list = []
    columns_name_list = data_frame.columns
    for i in columns_name_list:
        column_name = i
        this_trace = trace(data_frame.index, data_frame[column_name], column_name, chosen_mode)
        trace_list.append(this_trace)
    return trace_list


def connect_traces(traces1, trace2):
    connected_traces = []
    for i in traces1:
        connected_traces.append(i)
    for j in trace2:
        connected_traces.append(j)
    return connected_traces

def plot_bar(data_x, data_y, name):

    bar_trace = go.Bar(
        x= data_x,
        y= data_y,
        name= name,
        text = data_x,
        textposition='outside',
    )
    
    data = [ bar_trace]
    layout = dict(title=name, barmode='group',    
                  font=dict(size=16),
                 margin=dict(b=275),
                 xaxis=go.layout.XAxis(tickangle=35))
    fig = dict(data=data, layout=layout)
    
    html_name = name + '.html'
    plot(fig, filename=html_name)

In [ ]:
current_abs_path = os.getcwd()

Загрузим файлик с данными из текущей директории и посмотрим на него. При необходимости нужно менять его название

In [ ]:
abs_path_to_file = current_abs_path +'\\' +  file_name_with_regime_data
file = pd.read_csv(abs_path_to_file, sep = ";", header = None)
file.head()

Произведем предварительную обработку и вновь посмотрим на файлик

In [ ]:
file = edit_new_trd_file(file)
file.head()

Построим графики для исходных данных

In [ ]:
data_traces = create_traces_list_for_all_columms(file)
data = data_traces
file_name = "Показания датчиков исходные"
plot_func(file_name , file_name + ".html")

Определим интересующий промежуток времени для анализа. Если весь промежуток нормальный, можно поставить 1900 и 2200 год соответственно для выдачи всех данных

In [ ]:
year_start = 2009
month_start = 8
day_start = 15
hour_start = 20
minute_start = 6
second_start = 0


year_end = 2029
month_end = 8
day_end = 15
hour_end = 20
minute_end = 11
second_end = 22

date_from = pd.to_datetime(pd.DataFrame({'year': [year_start, year_end],
                           'month': [month_start, month_end],
                           'day': [day_start, day_end],
                            'hour': [hour_start, hour_end],
                                        'minute': [minute_start, minute_end],
                                        'second': [second_start, second_end]}))
date_start = date_from.iloc[0]
print("Дата начала периода для тщательного анализа: " + str(date_start))
date_end = date_from.iloc[1]
print("Дата конца периода для тщательного анализа: " + str(date_end))

Вырежем кусок исходных данным, посмотрим на него и продолжим работу только над ним

In [ ]:
nedeed_data = file[(file.index >= date_start) & (file.index <= date_end)]
nedeed_data.head()

Сделаем усреднение нужных данных и построим графики. Усреднение можно менять, например "1min", "2h"

In [ ]:
#resample_time_str = "30s"
#resampled_data = nedeed_data.resample(resample_time_str).mean()
#data_traces = create_traces_list_for_all_columms(resampled_data)
#data = data_traces
#file_name = "Показания датчиков с осреднением " + resample_time_str
#plot_func(file_name , file_name + ".html")


Построение отдельных графиков (по 8 штук) для сырых данных

In [ ]:
##Построим первые 10 графиков
#\traces_8_first = data_traces[0:8]
#traces_2_last = data_traces[-3:]
#traces_for_first_plot = connect_traces(traces_8_first, traces_2_last)
#data = traces_for_first_plot
#file_name = "Показания датчиков (первые 8 и тензо) с осреднением  " + resample_time_str
#plot_func(file_name , file_name + ".html")
##Еще вторые 8
#traces_8_second = data_traces[8:16]
#data = traces_8_second
#file_name = "Показания датчиков (вторые 8) с осреднением  " + resample_time_str
#plot_func(file_name , file_name + ".html")
##Еще третьи 8
#traces_8_third = data_traces[16:24]
#data = traces_8_third
#file_name = "Показания датчиков (третьи 8) с осреднением  " + resample_time_str
#plot_func(file_name , file_name + ".html")
##И последние 8
#traces_8_fourth = data_traces[16:24]
#data = traces_8_fourth
#file_name = "Показания датчиков (четвертые 8) с осреднением  " + resample_time_str
#plot_func(file_name , file_name + ".html")

Загрузим данные с датчиков в режиме простоя и вычтем их из интересующих данных для их калибровки

In [ ]:
abs_path_to_calibr_file = current_abs_path +'\\' +  file_name_with_corr_data
calibr_file = pd.read_csv(abs_path_to_calibr_file, sep = ";", header = None)
calibr_file = edit_new_trd_file(calibr_file)

#построение данных для калибровки
data_traces = create_traces_list_for_all_columms(calibr_file)
data = data_traces
file_name = "Показания датчиков для калибровки"
plot_func(file_name , file_name + ".html")

calibr_mean_values = calibr_file.mean()
nedeed_mean_values = nedeed_data.mean()
ready_mean_values = nedeed_mean_values - calibr_mean_values
ready_mean_values.iloc[-1] = calibr_mean_values.iloc[-1]
ready_mean_values[0:3] = nedeed_mean_values[0:3]

#ready_mean_values

Преобразуем сигналы в замеры согласно их спецификациям (должны быть определены в загружаемой таблице)

In [ ]:
description_file = pd.read_excel("Описание датчиков.xlsx")
#description_file.head()


row_val = ready_mean_values

min_val = description_file['Нижний предел измерения сенсора'].values
max_val = description_file['Верхний предел измерения датчика'].values
multiplier = description_file['Множитель для переводной формулы'].values

ready_data = func_to_row_data(min_val, max_val, row_val, multiplier)
#print(ready_data)

Построим некоторые графики

In [ ]:
plot_bar(description_file['Правильное название датчика'], ready_data, 'Первый результат построения данных эксперимента')

In [ ]:
stages_number = [0,1,2,3,5,7,9,10,12,14,17,19,20,22,25,29,33,37,38,58,77,78,98,117,118,119]
only_stage_data_array = np.asarray(ready_data[8:32].values)
input_pressure = np.asarray(ready_data[6])
output_pressure = np.asarray(ready_data[7])
all_esp_pressure = np.concatenate(([input_pressure],only_stage_data_array, [output_pressure]), axis=0)
stages_trace = trace(stages_number, all_esp_pressure, "Распределение давления по ступеням, бар")
data = [stages_trace]
file_name = "Распределение давления по ступеням, бар"
plot_func(file_name , file_name + ".html")

Сохраним результаты в ecxel

In [ ]:
ready_data.index = description_file['Правильное название датчика']
ready_data.columns = ["Величина параметра"]
excel_name = "Результаты обработки данных на текущем режиме_" + changed_mark + ".xlsx"
transposed_ready_data = pd.DataFrame(np.zeros((1,34)))
transposed_ready_data.iloc[0] = ready_data.values
transposed_ready_data.columns = description_file['Правильное название датчика']
transposed_ready_data.to_excel(excel_name)